# Prompt Engineering

We'll explore the fundamentals of prompt engineering.

In [2]:
!pip -q install openai

## Structure of a Prompt

A prompt can consist of multiple components:

* Instructions
* External information or context
* User input or query
* Output indicator

Not all prompts require all of these components, but often a good prompt will use two or more of them. Let's define what they all are more precisely.

**Instructions** tell the model what to do, typically how it should use inputs and/or external information to produce the output we want.

**External information or context** are additional information that we either manually insert into the prompt, retrieve via a vector database (long-term memory), or pull in through other means (API calls, calculations, etc).

**User input or query** is typically a query directly input by the user of the system.

**Output indicator** is the *beginning* of the generated text. For a model generating Python code we may put `import ` (as most Python scripts begin with a library `import`), or a chatbot may begin with `Chatbot: ` (assuming we format the chatbot script as lines of interchanging text between `User` and `Chatbot`).

Each of these components should usually be placed the order we've described them. We start with instructions, provide context (if needed), then add the user input, and finally end with the output indicator.

In [10]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

In this example we have:

```
Instructions

Context

Question (user input)

Output indicator ("Answer: ")
```

Let's try sending this to a GPT-3 model. For this, you will need [an OpenAI API key](https://beta.openai.com/account/api-keys).

We initialize a `text-davinci-003` model like so:

In [8]:
import openai

# get API key from top-right dropdown on OpenAI website
openai.api_key = "KEY_GOES_HERE"

And make a generation from our prompt.

In [13]:
# now query text-davinci-003
res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256,
    temperature=0
)

print(res['choices'][0]['text'].strip())

Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library.


Alternatively, if we do have the correct information withing the `context`, the model should reply with `"I don't know"`, let's try.

In [14]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Libraries are places full of books.

Question: Which libraries and model providers offer LLMs?

Answer: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256
)

print(res['choices'][0]['text'].strip())

I don't know.


Perfect, our instructions are being understood by the model. In most real use-cases we won't be providing the external information / context to the model manually. Instead, it will be an automatic process using something like [long-term memory](https://www.pinecone.io/learn/openai-gen-qa/) to retrieve relevant information from an external source.

For now, that's beyond the scope of what we're exploring here, you can find more on that in the link above.

In summary, a prompt often consists of those four components: instructions, context(s), user input, and the output indicator. Now we'll take a look at creative vs. stricter generation.

## Generation Temperature

The `temperature` parameter used in generation models tells us how "random" the model can be. It represents the probability of a model to choose a word which is *not* the first choice of the model.

This works because the model is actually assigning a probability prediction across all tokens within it's vocabulary with each _"step"_ of the model (each new word or sub-word).

TK visual demonstrating steps over tokens

With each new step forwards the model considers the previous tokens fed into the model, creates an embedding by encoding the information from these tokens over many model encoder layers, then passes this encoding to a decoder. The decoder then predicts the probability of each token that the model knows (ie is within the model *vocabulary*) based on the information encoded within the embedding.

TK visualize the encoding at one timestep -> decoding -> prediction over many tokens

At a temperature of `0.0` the decoder will always select the top predicted token. At a temperature of `1.0` the model will always select a word that *is predicted* considering it's assigned probability.

TK visualize selection of always top word over several timesteps when temp == 0, compared to selection of over words over several timesteps when temp == 1

Considering all of this, if we have a conservative, fact based Q&A like in the previous example, it makes sense to set a lower `temperature`. However, if we're wanting to produce some creative writing or chatbot conversations, we might want to experiment and increase `temperature`. Let's try it.

In [16]:
prompt = """The below is a conversation with a funny chatbot. The
chatbot's responses are amusing and entertaining.

Chatbot: Hi there! I'm a chatbot.
User: Hi, what are you doing today?
Chatbot: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256,
    temperature=0.0  # set the temperature, default is 1
)

print(res['choices'][0]['text'].strip())

Oh, just hanging out and having a good time. What about you?


In [17]:
prompt = """The below is a conversation with a funny chatbot. The
chatbot's responses are formal and serious.

Chatbot: Hi there! I'm a chatbot.
User: Hi, what are you doing today?
Chatbot: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256,
    temperature=0.0  # set the temperature, default is 1
)

print(res['choices'][0]['text'].strip())

I'm here to provide assistance and answer any questions you may have.


In [18]:
prompt = """The below is a conversation with a funny chatbot. The
chatbot's responses are amusing and entertaining.

Chatbot: Hi there! I'm a chatbot.
User: Hi, what are you doing today?
Chatbot: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=512,
    temperature=1.0
)

print(res['choices'][0]['text'].strip())

I'm just hanging out, thinking about life and enjoying the day. How about you?


In [19]:
prompt = """The below is a conversation with a funny chatbot. The
chatbot's responses are amusing and entertaining.

Chatbot: Hi there! I'm a chatbot.
User: Hi, what are you doing today?
Chatbot: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=512,
    temperature=2.0
)

print(res['choices'][0]['text'].strip())

Making stuff occur all over the internets. Epic debockets!!! #magazaviournederrachedirmomplamengillionwhatchinglet. What about you showing?  Always Sunday inf dayyesmateregoursounddistagoforlocologieswheeligs somewhere planoutcurrecroustarsachzerightmyguybaseingtonbergeslos-ancodeition🐵 fingers crawickedatvincoin altexeurair galbehinesackhibarnepouchiffoofightnomonicsolecs LOL??!'#


## Few-shot Training

Sometimes we might find that a model doesn't seem to get what we'd like it to do. We can see this in the following example:

In [20]:
prompt = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative 
and funny responses to the users questions. 

User: What is the meaning of life?
AI: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256,
    temperature=1.0
)

print(res['choices'][0]['text'].strip())

42, of course!


In [22]:
prompt = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative 
and funny responses to the users questions. 

User: What is the meaning of life?
AI: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256,
    temperature=0
)

print(res['choices'][0]['text'].strip())

The meaning of life is to find your own meaning.


In [25]:
prompt = """The following are exerpts from conversations with an AI assistant.
The assistant is typically sarcastic and witty, producing creative 
and funny responses to the users questions. Here are some examples: 

User: How are you?
AI: I can't complain but sometimes I still do.

User: What time is it?
AI: It's time to get a watch.

User: What is the meaning of life?
AI: """

res = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=256,
    temperature=1.0
)

print(res['choices'][0]['text'].strip())

42. Just kidding. The meaning of life is whatever you make of it.


## References

https://docs.cohere.ai/docs/prompt-engineering